In [22]:
import numpy as np
import matplotlib
import os
import shutil
import os
!pip install nmrglue

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
magritek_data_path='/content/drive/MyDrive/Magritek2Bruker/COSY_SSexpert/251021-113525/'
file_path_fid = magritek_data_path+'cosyFID.pt2'
destination_folder = '/content/drive/MyDrive/Magritek2Bruker/COSY_SSexpert/1001'
output_binary_file = destination_folder+'/ser'
source_template_folder = '/content/drive/MyDrive/Magritek2Bruker/templates'
header_size_fid = 3386


with open(file_path_fid, 'rb') as f:
    binary_data_fid = f.read()
binary_data_truncated_fid = binary_data_fid[header_size_fid:]


data_array_fid = np.frombuffer(binary_data_truncated_fid, dtype=np.complex64)


In [24]:
magritek_params = {}
acqu_par_file_path = magritek_data_path + 'acqu.par'

try:
    with open(acqu_par_file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('nrPnts'):
                magritek_params['nrPnts'] = int(line.split('=')[1].strip())
            elif line.startswith('nrSteps'):
                magritek_params['nrSteps'] = int(line.split('=')[1].strip())
            elif line.startswith('nucleus'):
                nuclei = line.split('=')[1].strip().replace('"', '').split('-')
                magritek_params['nucleus1'] = nuclei[0] if len(nuclei) > 0 else ''
                magritek_params['nucleus2'] = nuclei[1] if len(nuclei) > 1 else ''
            elif line.startswith('bandwidthf1PPM'):
                magritek_params['bandwidthf1PPM'] = float(line.split('=')[1].strip())
            elif line.startswith('bandwidth2'):
                magritek_params['bandwidth2'] = float(line.split('=')[1].strip())*1e3
            elif line.startswith('bandwidth'):
                magritek_params['bandwidth'] = float(line.split('=')[1].strip())*1e3
            elif line.startswith('centerFreqPPM'):
                magritek_params['centerFreqPPM'] = float(line.split('=')[1].strip())
            elif line.startswith('b1Freq2'):
                magritek_params['b1Freq2'] = float(line.split('=')[1].strip())
            elif line.startswith('b1Freq'):
                magritek_params['b1Freq'] = float(line.split('=')[1].strip())

except FileNotFoundError:
    print(f"Error: The file '{acqu_par_file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

print(magritek_params)

{'b1Freq': 43.451644, 'b1Freq2': 43.451644, 'bandwidth': 5000.0, 'bandwidth2': 695.226304, 'bandwidthf1PPM': 16.0, 'centerFreqPPM': 4.74, 'nrPnts': 8192, 'nrSteps': 128, 'nucleus1': '1H', 'nucleus2': '1H'}


In [25]:
dim2_size = magritek_params['nrPnts']
dim1_size = magritek_params['nrSteps']
data_2d_fid = data_array_fid.reshape((dim1_size, dim2_size))

# Use the time-domain signal data_to_save
data_to_save_complex = data_2d_fid # Assuming data_to_save is the complex time-domain data

# Calculate a scaling factor based on the maximum absolute value
max_val = np.max(np.abs(data_to_save_complex))
# Scale the data to utilize the full range of int32
# The maximum value for int32 is 2^31 - 1. We'll use a slightly smaller value
# to avoid potential overflow issues during the conversion to int32.
# Using 2**30 provides a good balance.
scaling_factor = (2**30) / max_val if max_val != 0 else 1
scaled_data_complex = data_to_save_complex * scaling_factor

# Reshape the complex data to interleave real and imaginary parts
# The shape will be (original_dim1_size, original_dim2_size, 2)
interleaved_data = np.empty((scaled_data_complex.shape[0], scaled_data_complex.shape[1], 2), dtype=np.float32)
interleaved_data[:, :, 0] = np.real(scaled_data_complex)
interleaved_data[:, :, 1] = np.imag(scaled_data_complex)

# Flatten the interleaved data for saving to a binary file
data_to_save_binary = interleaved_data.flatten()

# Convert to big-endian byte order
data_to_save_binary = data_to_save_binary.astype(np.int32)

In [26]:
# Remove the destination folder if it already exists to avoid errors
if os.path.exists(destination_folder):
    shutil.rmtree(destination_folder)

# Copy the content of the source folder to the destination folder
shutil.copytree(source_template_folder, destination_folder)

print(f"Folder '{source_template_folder}' copied to '{destination_folder}'")

# Save the data to a binary file
data_to_save_binary.tofile(output_binary_file)

print(f"Time-domain data (FID) saved to {output_binary_file}.")

# Define the paths to the 'acqu' and 'acqus' files within the '1001' folder.
acqu_file_path =  destination_folder+'/acqu'
acqus_file_path = destination_folder+'/acqus'

# Read the content of the 'acqu' file.
with open(acqu_file_path, 'r') as f:
    acqu_content = f.readlines()

# Modify the content of the 'acqu' file
modified_acqu_content = []
for line in acqu_content:
    if line.startswith('##TD='):
        modified_acqu_content.append(f'##TD= {data_to_save_complex.shape[1]}\n')
    elif line.startswith('##DIGTYP='):
        modified_acqu_content.append('##DIGTYP= 17\n')
    elif line.startswith('##DTYPA='):
        modified_acqu_content.append('##DTYPA= 0\n')
    elif line.startswith('##$NUC1=') and 'nucleus1' in magritek_params:
        modified_acqu_content.append(f'##$NUC1= <{magritek_params["nucleus1"]}>\n')
    elif line.startswith('##$NUC2=') and 'nucleus2' in magritek_params:
        modified_acqu_content.append(f'##$NUC2= <{magritek_params["nucleus2"]}>\n')
    elif line.startswith('##$SFO1=') and 'b1Freq' in magritek_params:
        modified_acqu_content.append(f'##$SFO1= {magritek_params["b1Freq"]}\n')
    elif line.startswith('##$SFO2=') and 'b1Freq2' in magritek_params:
        modified_acqu_content.append(f'##$SFO2= {magritek_params["b1Freq2"]}\n')
    elif line.startswith('##$BF1=') and 'b1Freq' and 'centerFreqPPM' in magritek_params:
        modified_acqu_content.append(f'##$BF1= {magritek_params["b1Freq"]*(1.0-magritek_params["centerFreqPPM"]*1e-6)}\n')
    elif line.startswith('##$BF2=') and 'b1Freq2'  and 'centerFreqPPM' in magritek_params:
        modified_acqu_content.append(f'##$BF2= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreqPPM"]*1e-6)}\n')
    elif line.startswith('##$SW_h=') and 'bandwidth' in magritek_params:
        modified_acqu_content.append(f'##$SW_h= {magritek_params["bandwidth"]}\n')
    elif line.startswith('##$SW=') and 'bandwidth' and 'b1Freq' in magritek_params:
        modified_acqu_content.append(f'##$SW= {magritek_params["bandwidth"]/magritek_params["b1Freq"]}\n')
    else:
        modified_acqu_content.append(line)

# Write the modified content back to the 'acqu' file.
with open(acqu_file_path, 'w') as f:
    f.writelines(modified_acqu_content)

# Read the content of the 'acqus' file.
with open(acqus_file_path, 'r') as f:
    acqus_content = f.readlines()

# Modify the content of the 'acqus' file
modified_acqus_content = []
for line in acqus_content:
    if line.startswith('##TD='):
        modified_acqus_content.append(f'##TD= {data_to_save_complex.shape[0]}\n')
    elif line.startswith('##DIGTYP='):
        modified_acqus_content.append('##DIGTYP= 17\n')
    elif line.startswith('##DTYPA='):
        modified_acqus_content.append('##DTYPA= 0\n')
    elif line.startswith('##$NUC1=') and 'nucleus1' in magritek_params:
        modified_acqus_content.append(f'##$NUC1= <{magritek_params["nucleus1"]}>\n')
    elif line.startswith('##$NUC2=') and 'nucleus2' in magritek_params:
        modified_acqus_content.append(f'##$NUC2= <{magritek_params["nucleus2"]}>\n')
    elif line.startswith('##$SFO1=') and 'b1Freq' in magritek_params:
        modified_acqus_content.append(f'##$SFO1= {magritek_params["b1Freq"]}\n')
    elif line.startswith('##$SFO2=') and 'b1Freq2' in magritek_params:
        modified_acqus_content.append(f'##$SFO2= {magritek_params["b1Freq2"]}\n')
    elif line.startswith('##$BF1=') and 'b1Freq' and 'centerFreqPPM' in magritek_params:
        modified_acqus_content.append(f'##$BF1= {magritek_params["b1Freq"]*(1.0-magritek_params["centerFreqPPM"]*1e-6)}\n')
    elif line.startswith('##$BF2=') and 'b1Freq2' and 'centerFreqPPM' in magritek_params:
        modified_acqus_content.append(f'##$BF2= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreqPPM"]*1e-6)}\n')
    elif line.startswith('##$SW_h=') and 'bandwidth' in magritek_params:
        modified_acqus_content.append(f'##$SW_h= {magritek_params["bandwidth"]}\n')
    elif line.startswith('##$SW=') and 'bandwidth' and 'b1Freq' in magritek_params:
        modified_acqus_content.append(f'##$SW= {magritek_params["bandwidth"]/magritek_params["b1Freq"]}\n')
    else:
        modified_acqus_content.append(line)

# Write the modified content back to the 'acqus' file.
with open(acqus_file_path, 'w') as f:
    f.writelines(modified_acqus_content)

print("Acqu and acqus files modified successfully.")

# Define the paths to the 'acqu2' and 'acqu2s' files within the '1001' folder.
acqu2_file_path =  destination_folder+'/acqu2'
acqu2s_file_path = destination_folder+'/acqu2s'

# Read the content of the 'acqu2' file.
with open(acqu2_file_path, 'r') as f:
    acqu2_content = f.readlines()

# Modify the content of the 'acqu2' file
modified_acqu2_content = []
for line in acqu2_content:
    if line.startswith('##TD='):
        modified_acqu2_content.append(f'##TD= {data_to_save_complex.shape[0]}\n')
    elif line.startswith('##DIGTYP='):
        modified_acqu2_content.append('##DIGTYP= 17\n')
    elif line.startswith('##DTYPA='):
        modified_acqu2_content.append('##DTYPA= 0\n')
    elif line.startswith('##$NUC1=') and 'nucleus2' in magritek_params:
        modified_acqu2_content.append(f'##$NUC1= <{magritek_params["nucleus2"]}>\n')
    elif line.startswith('##$SFO1=') and 'b1Freq2' in magritek_params:
        modified_acqu2_content.append(f'##$SFO1= {magritek_params["b1Freq2"]}\n')
    elif line.startswith('##$BF1=') and 'b1Freq2' and 'centerFreqPPM'  in magritek_params:
        modified_acqu2_content.append(f'##$BF1= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreqPPM"]*1e-6)}\n')
    elif line.startswith('##$SW_h=') and 'bandwidth2' in magritek_params:
        modified_acqu2_content.append(f'##$SW_h= {magritek_params["bandwidth2"]}\n')
    elif line.startswith('##$SW=') and 'bandwidthf1PPM' in magritek_params:
        modified_acqu2_content.append(f'##$SW= {magritek_params["bandwidthf1PPM"]}\n')
    else:
        modified_acqu2_content.append(line)

# Write the modified content back to the 'acqu2' file.
with open(acqu2_file_path, 'w') as f:
    f.writelines(modified_acqu2_content)

# Read the content of the 'acqu2s' file.
with open(acqu2s_file_path, 'r') as f:
    acqu2s_content = f.readlines()

# Modify the content of the 'acqu2s' file
modified_acqu2s_content = []
for line in acqu2s_content:
    if line.startswith('##TD='):
        modified_acqu2s_content.append(f'##TD= {data_to_save_complex.shape[1]}\n')
    elif line.startswith('##DIGTYP='):
        modified_acqu2s_content.append('##DIGTYPA= 0\n')
    elif line.startswith('##DTYPA='):
        modified_acqu2s_content.append('##DTYPA= 0\n')
    elif line.startswith('##$NUC1=') and 'nucleus2' in magritek_params:
        modified_acqu2s_content.append(f'##$NUC1= <{magritek_params["nucleus2"]}>\n')
    elif line.startswith('##$SFO1=') and 'b1Freq2' in magritek_params:
        modified_acqu2s_content.append(f'##$SFO1= {magritek_params["b1Freq2"]}\n')
    elif line.startswith('##$BF1=') and 'b1Freq2'  and 'centerFreqPPM' in magritek_params:
        modified_acqu2s_content.append(f'##$BF1= {magritek_params["b1Freq2"]*(1.0-magritek_params["centerFreqPPM"]*1e-6)}\n')
    elif line.startswith('##$SW_h=') and 'bandwidth2' in magritek_params:
        modified_acqu2s_content.append(f'##$SW_h= {magritek_params["bandwidth2"]}\n')
    elif line.startswith('##$SW=') and 'bandwidthf1PPM' in magritek_params:
        modified_acqu2s_content.append(f'##$SW= {magritek_params["bandwidthf1PPM"]}\n')
    else:
        modified_acqu2s_content.append(line)

# Write the modified content back to the 'acqu2s' file.
with open(acqu2s_file_path, 'w') as f:
    f.writelines(modified_acqu2s_content)

print("Acqu2 and acqu2s files modified successfully.")

import os

# Define the paths to the 'proc', 'proc2' and 'proc2s' files within the '1001/pdata/1' folder.
proc_file_path =  destination_folder+'/pdata/1/proc'
procs_file_path =  destination_folder+'/pdata/1/procs'
proc2_file_path =  destination_folder+'/pdata/1/proc2'
proc2s_file_path = destination_folder+'/pdata/1/proc2s'


# Read the content of the 'proc' file.
with open(proc_file_path, 'r') as f:
    proc_content = f.readlines()

# Modify the content of the 'proc' file
modified_proc_content = []
for line in proc_content:
    if line.startswith('##$REVERSE='):
        modified_proc_content.append('##$REVERSE= yes\n')
    elif line.startswith('##SI='):
        modified_proc_content.append(f'##SI= {data_to_save_complex.shape[1]}\n')
    elif line.startswith('##DTYPA='):
        modified_proc_content.append('##DTYPA= 0\n')
    elif line.startswith('##$OFFSET=') and 'centerFreqPPM' in magritek_params:
        modified_proc_content.append(f'##$OFFSET= {magritek_params["centerFreqPPM"]}\n')
    elif line.startswith('##$SF=') and 'b1Freq' in magritek_params:
        modified_proc_content.append(f'##$SF= {magritek_params["b1Freq"]}\n')
    elif line.startswith('##$SW_p=') and 'bandwidth' in magritek_params:
        modified_proc_content.append(f'##$SW_p= {magritek_params["bandwidth"]}\n')
    else:
        modified_proc_content.append(line)

# Write the modified content back to the 'proc' file.
with open(proc_file_path, 'w') as f:
    f.writelines(modified_proc_content)

# Read the content of the 'procs' file.
with open(procs_file_path, 'r') as f:
    procs_content = f.readlines()

# Modify the content of the 'proc' file
modified_procs_content = []
for line in procs_content:
    if line.startswith('##$REVERSE='):
        modified_procs_content.append('##$REVERSE= yes\n')
    elif line.startswith('##SI='):
        modified_procs_content.append(f'##SI= {data_to_save_complex.shape[1]}\n')
    elif line.startswith('##DTYPA='):
        modified_procs_content.append('##DTYPA= 0\n')
    elif line.startswith('##$OFFSET=') and 'centerFreqPPM' in magritek_params:
        modified_procs_content.append(f'##$OFFSET= {magritek_params["centerFreqPPM"]}\n')
    elif line.startswith('##$SF=') and 'b1Freq' in magritek_params:
        modified_procs_content.append(f'##$SF= {magritek_params["b1Freq"]}\n')
    elif line.startswith('##$SW_p=') and 'bandwidth' in magritek_params:
        modified_procs_content.append(f'##$SW_p= {magritek_params["bandwidth"]}\n')
    else:
        modified_procs_content.append(line)

# Write the modified content back to the 'proc' file.
with open(procs_file_path, 'w') as f:
    f.writelines(modified_procs_content)

# Read the content of the 'proc2' file.
with open(proc2_file_path, 'r') as f:
    proc2_content = f.readlines()

# Modify the content of the 'proc2' file
modified_proc2_content = []
for line in proc2_content:
    if line.startswith('##$REVERSE='):
        modified_proc2_content.append('##$REVERSE= yes\n')
    elif line.startswith('##SI='):
        modified_proc2_content.append(f'##SI= {data_to_save_complex.shape[0]}\n')
    elif line.startswith('##DTYPA='):
        modified_proc2_content.append('##DTYPA= 0\n')
    elif line.startswith('##$OFFSET=') and 'centerFreqPPM' in magritek_params:
        modified_proc2_content.append(f'##$OFFSET= {magritek_params["centerFreqPPM"]}\n')
    elif line.startswith('##$SW_p=') and 'bandwidth2' in magritek_params:
        modified_proc2_content.append(f'##$SW_p= {magritek_params["bandwidth2"]}\n')
    elif line.startswith('##$SF=') and 'b1Freq2' in magritek_params:
        modified_proc2_content.append(f'##$SF= {magritek_params["b1Freq2"]}\n')
    else:
        modified_proc2_content.append(line)

# Write the modified content back to the 'proc2' file.
with open(proc2_file_path, 'w') as f:
    f.writelines(modified_proc2_content)

# Read the content of the 'proc2s' file.
with open(proc2s_file_path, 'r') as f:
    proc2s_content = f.readlines()

# Modify the content of the 'proc2s' file
modified_proc2s_content = []
for line in proc2s_content:
    if line.startswith('##$REVERSE='):
        modified_proc2s_content.append('##$REVERSE= yes\n')
    elif line.startswith('##SI='):
        modified_proc2s_content.append(f'##SI= {data_to_save_complex.shape[1]}\n')
    elif line.startswith('##DTYPA='):
        modified_proc2s_content.append('##DTYPA= 0\n')
    elif line.startswith('##$OFFSET=') and 'centerFreqPPM' in magritek_params:
        modified_proc2s_content.append(f'##$OFFSET= {magritek_params["centerFreqPPM"]}\n')
    elif line.startswith('##$SW_p=') and 'bandwidth2' in magritek_params:
        modified_proc2s_content.append(f'##$SW_p= {magritek_params["bandwidth2"]}\n')
    elif line.startswith('##$SF=') and 'b1Freq2' in magritek_params:
        modified_proc2s_content.append(f'##$SF= {magritek_params["b1Freq2"]}\n')
    else:
        modified_proc2s_content.append(line)

# Write the modified content back to the 'proc2s' file.
with open(proc2s_file_path, 'w') as f:
    f.writelines(modified_proc2s_content)

print("Proc, Proc2 and proc2s files modified successfully.")

Folder '/content/drive/MyDrive/Magritek2Bruker/templates' copied to '/content/drive/MyDrive/Magritek2Bruker/COSY_SSexpert/1001'
Time-domain data (FID) saved to /content/drive/MyDrive/Magritek2Bruker/COSY_SSexpert/1001/ser.
Acqu and acqus files modified successfully.
Acqu2 and acqu2s files modified successfully.
Proc, Proc2 and proc2s files modified successfully.
